In [ ]:
# This cell is only for setting example parameter defaults - gets replaced by sidecar.
day_obs = "2024-12-10"
day_obs = "2024-08-13"
day_obs = "Today"
telescope = "AuxTel"  
#telescope = "SimonyiTel"
instrument = "latiss"  
#instrument = "lsstcomcam"
salindex = 2
#salindex = 1
timezone = "Chile/Continental"
#timezone = "UTC"

# Scheduler: Targets, Observations and ConsDB Visits for {{ params.day_obs }} {{ params.telescope }}

The Feature Based Scheduler requests `Targets` and completed observation scripts result in `Observations`; these are both are tracked in the EFD.
The Consdb reports acquired `Visits`.  
On-sky exposures can also be acquired directly through execution of scripts or JSON BLOCKs; these don't result in `Targets` but do produce `Visits`.


* [Almanac](#Almanac)
* [EFD Configuration](#EFD_configuration)
* [EFD Targets and Observations](#EFD_targets)
* [ConsDB Visits](#Consdb_visits)
* [Overheads and Issues](#Overheads)
* [Summary Plot](#Summary_plot)
* [Night Log Report](#Night_report)

In [ ]:
import os
import warnings
import copy
from math import floor
import numpy as np
import healpy as hp
import matplotlib.pylab as plt
from cycler import cycler
import colorcet as cc

import pandas as pd
from pandas import option_context
from IPython.display import display, Markdown, HTML

import datetime
from astropy.time import Time, TimeDelta
import astropy.units as u
import astropy
astropy.utils.iers.conf.iers_degraded_accuracy = 'ignore'

import datetime
import pytz

import datetime
from zoneinfo import ZoneInfo

try:
    tz = ZoneInfo(timezone)
    tz_utc = ZoneInfo("UTC")
except ZoneInfoNotFoundError:
    print("Timezone should be a string recognizable to `ZoneInfo`.")
    print("Using Chile/Continental (+UTC) backup.")
    tz = ZoneInfo("Chile/Continental")
    tz_utc = ZoneInfo("UTC")

from rubin_scheduler.site_models import Almanac
from rubin_scheduler.utils import Site

import requests
import urllib
import sqlalchemy
from lsst_efd_client import EfdClient
try:
    from lsst.summit.utils import ConsDbClient
    have_consdb = True
except ImportError:
    have_consdb = False

# Access nightlog and JIRA
try:
    from lsst.rsp import get_access_token
except ImportError:
    def get_access_token(token_file : str | None = None) -> str:
        token = os.environ.get("ACCESS_TOKEN")
        if token == None:
            if token_file is not None:
                with open("token_file", "r") as f:
                    token = f.read()
            else:
                warnings.warn("No RSP token available.")
        return token

# for scheduler snapshots
from urllib.parse import urlparse
from lsst.resources import ResourcePath


# at USDF or at summit?
if os.getenv("EXTERNAL_INSTANCE_URL", "") == "https://summit-lsp.lsst.codes":
    efd = 'summit_efd'
else:
    efd = 'usdf_efd' 
    if 'usdf' in os.getenv("EXTERNAL_INSTANCE_URL", ""):
        os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"
        os.environ["no_proxy"] += ",.consdb"
    
# Not sure of summit consdb access, just use USDF for now
os.environ["LSST_CONSDB_PQ_URL"] = "http://consdb-pq.consdb:8080/consdb"

%matplotlib inline

In [ ]:
if day_obs.lower() == "today":
    # Shift the 12hour offset following the definition of day_obs in https://sitcomtn-032.lsst.io/    
    # Drop the hours, minutes, seconds to get the ISO formatted day_obs
    day_obs = Time(np.floor(Time.now().mjd - 0.5), format='mjd', scale='utc').iso[0:10]

elif day_obs.lower() == "yesterday":
    # Shift the 12hour offset following the definition of day_obs in https://sitcomtn-032.lsst.io/
    # Drop the hours, minutes, seconds to get the ISO fromatted day_obs
    day_obs = (Time(np.floor(Time.now().mjd - 0.5), format='mjd', scale='utc') - TimeDelta(1, format='jd')).iso[0:10]

else:
    # test that day_obs is a proper day_obs
    try:
        test_day_obs = Time(f"{day_obs}T12:00:00", format='isot', scale='utc')
    except ValueError:
        msg = "day_obs should be a date formatted as YYYY-MM-DD"
        raise ValueError(msg)

In [ ]:
minutes_to_days = 1./60/24
seconds_to_days = 1./60/60/24

<a id="Almanac"></a>

In [ ]:
## Almanac ## 

display(Markdown(f"## Almanac information for dayobs {day_obs}"))
site = Site('LSST')
almanac = Almanac()
night_events = almanac.get_sunset_info(evening_date=day_obs, longitude=site.longitude_rad)
civil_sunset = Time(night_events['sunset'], format='mjd', scale='utc') 
sunset = Time(night_events['sun_n12_setting'], format='mjd', scale='utc') 
sunrise = Time(night_events['sun_n12_rising'], format='mjd', scale='utc')
night_length = sunrise.mjd - sunset.mjd

display(Markdown(f"12-deg sunset at {sunset.to_datetime(timezone=tz_utc).strftime('%x %X')} UTC  -- {sunset.to_datetime(timezone=tz).strftime('%x %X')} {timezone}"))
display(Markdown(f"12-deg sunrise at {sunrise.to_datetime(timezone=tz_utc).strftime('%x %X')} UTC --  {sunrise.to_datetime(timezone=tz).strftime('%x %X')} {timezone}"))
display(Markdown(f"allowing for a night of {night_length * 24 :.2f} hours"))
moon_phase = almanac.get_sun_moon_positions(sunset.mjd)['moon_phase']
if not np.isnan(night_events['moonrise']):
    moonrise = Time(night_events['moonrise'], format='mjd', scale='utc')
    display(Markdown(f"Moonrise is at {moonrise.to_datetime(timezone=tz_utc).strftime('%x %X')} UTC -- {moonrise.to_datetime(timezone=tz).strftime('%x %X')} {timezone}"))
if not np.isnan(night_events['moonset']):
    moonset = Time(night_events['moonset'], format='mjd', scale='utc')
    display(Markdown(f"Moonset at {moonset.to_datetime(timezone=tz_utc).strftime('%x %X')} UTC -- {moonset.to_datetime(timezone=tz).strftime('%x %X')} {timezone}"))
display(Markdown(f"Moon phase is {moon_phase :.1f} (0=new, 100=full)."))

In [ ]:
# Report time notebook was run, which is likely useful if running notebook in the middle or start of the night
current_time = Time.now()
display(Markdown(f"Time of notebook execution: {current_time.isot}"))
if current_time > sunrise:
    display(Markdown("Night is complete."))
elif current_time < sunrise and current_time > civil_sunset:
    display(Markdown("Night is in progress."))
elif current_time < civil_sunset:
    display(Markdown("Night not yet started."))

In [ ]:
efd_client = EfdClient(efd)
obsenv_client = EfdClient(efd, db_name='lsst.obsenv')

<a id="EFD_configuration"></a>

## EFD Configuration Information

In [ ]:
# Look at civil sunset, in case of early use of scheduler
early_setup = civil_sunset

#display(Markdown(f"## EFD Information")) 

# What versions of the Scheduler modules are being used
display(Markdown("### Scheduler Versions"))

# Scheduler dependency information
async def get_scheduler_configs(t_start: Time, t_end: Time, efd_client: EfdClient, obsenv_client: EfdClient, queueIndex: int | None =None) -> pd.DataFrame:
    """Return information needed to recreate FBS configuration. 

    This requires checking the obsenv (`lsst.obsenv.summary`) to find the version of ts_config_ocs in use, 
    the EFD (`lsst.sal.Scheduler.logevent_dependenciesVersions`) to find the version of rubin_scheduler and dependencies,
    and the EFD (`lsst.sal.Scheduler.logevent_configureApplied`) to find the specific FBS configuration file in use.

    Searches both the time within t_start to t_end, as well as the last configuration applied before this time period. 
    
    Defining queueIndex will search dependencies and configurations for that queue only. 

    Parameters
    ----------
    t_start : `astropy.Time`
        The time of the start of the period. 
    t_end : `astropy.Time`
        The time at the end of the period.
    efd_client : `lsst.efd.EfdClient`
        An EFD client pointed to the standard EFD database.
    obsenv_client : `lsst.efd.EfdClient`
        An EFD client pointed to the obsenv database.
    queueIndex : `int` or `None`
        The salIndex of a specific queue (1=Simonyi, 2=Auxtel, 3=OCS).
        If None, queries all queues, but the initial state may be missed.

    Returns
    -------
    sched_config : `pd.DataFrame`
        A dataframe carrying the configuration information.
        Some columns are compacted into single strings, so 
        the entire dataframe can fit into a limited set of columns.
    """
    # First find the obsenv to find the version of ts_config_ocs
    topic = 'lsst.obsenv.summary'
    fields = ['summit_extras', 'summit_utils',  'ts_standardscripts', 'ts_externalscripts', 'ts_config_ocs']
    obsenv_start = await obsenv_client.select_top_n(topic, fields, num=1, time_cut=t_start.isot)
    obsenv = await obsenv_client.select_time_series(topic, fields, t_start, t_end)
    obsenv = pd.concat([obsenv_start, obsenv])
    if len(obsenv) == 0:
        warnings.warn("Could not find obsenv values.")
        # This shouldn't happen, but could before obsenv was implemented.
        # We need something to fill in for work below.
        bad_obsenv0 = [(t_start - TimeDelta(1, format='mjd') * 3).utc.datetime] + ['unknown' for f in fields]
        bad_obsenv1 = [t_start.utc.datetime] + ['unknown' for f in fields]
        obsenv = pd.DataFrame([bad_obsenv0, bad_obsenv1], columns=['time'] + fields)
        obsenv.set_index('time', inplace=True)
        obsenv.index = obsenv.index.tz_localize("UTC")

    # Label whether it was an obsenv *update* (i.e. changed ts_config_ocs, etc)
    # Or just an obsenv *check* without update 
    # (obsenv entries are triggered by a command at the summit, which could be either of these jobs)
    check = np.all((obsenv[fields][1:].values == obsenv[fields][:-1].values), axis=1)
    classname = np.where(check, "Obsenv Check", "Obsenv Update")
    obsenv['classname'] = np.concatenate([np.array(['Obsenv']), classname])
    # Reconfigure some of the values to match the dataframe shape for logs 
    obsenv['description'] = ("ts_config_ocs: " + obsenv['ts_config_ocs'])
    # Build compact config string
    obsenv['config'] = ("ts_standardscripts: " + obsenv['ts_standardscripts'] + 
                        "; ts_externalscripts: " + obsenv['ts_externalscripts'] + 
                        "; summit_utils: " + obsenv['summit_utils'] + 
                        "; summit_extras: " + obsenv['summit_extras'])
    # The obsenv is shared across all scriptqueues. The salIndex has to apply to all.
    obsenv['salIndex'] = 0
    obsenv['script_salIndex'] = -1

    # Scheduler dependency information - updated independently of obsenv.
    topic = 'lsst.sal.Scheduler.logevent_dependenciesVersions'
    fields = ['cloudModel', 'downtimeModel', 'seeingModel', 'skybrightnessModel', 'observatoryLocation', 'observatoryModel', 'scheduler', 'salIndex', 'version']
    deps_start = await efd_client.select_top_n(topic, fields, num=1, time_cut=t_start.isot, index=queueIndex)  
    deps = await efd_client.select_time_series(topic, fields, t_start, t_end, index=queueIndex)    
    deps = pd.concat([deps_start, deps])
    if len(deps) == 0:
        warnings.warn("Could not find scheduler dependencies.")
        bad_deps = [t_start.utc.datetime] + ['unknown' for f in fields]
        deps = pd.DataFrame(bad_deps, columns=['time'] + fields)
        deps.set_index('time', inplace=True)
        deps.index = deps.index.tz_localize("UTC")
    
    # Reconfigure output to fit into script_status fields 
    deps['classname'] = "Scheduler dependencies"
    deps['description'] = f"{deps['scheduler'].values[0]} {deps['seeingModel'].values[0]}"
    models = [c for c in deps.columns if 'observatory' in c or 'Model' in c]
    def build_compact_config_string(x, models): 
        dep_string = ''
        for m in models:
            dep_string += f"{m}: {x[m]}, "
        dep_string = dep_string[:-2]
        return dep_string
    deps['config'] = deps.apply(build_compact_config_string, args=[models], axis=1)
    deps['script_salIndex'] = -1
    
    # The configurationApplied should happen with every scheduler update
    topic = 'lsst.sal.Scheduler.logevent_configurationApplied'
    fields = ['SchedulerId', 'configurations', 'salIndex', 'schemaVersion', 'url', 'version']
    conf_start = await efd_client.select_top_n(topic, fields, num=1, time_cut=t_start.isot, index=queueIndex)
    conf = await efd_client.select_time_series(topic, fields, t_start, t_end, index=queueIndex)
    conf = pd.concat([conf_start, conf])
    if len(conf) == 0:
        warnings.warn("Could not find scheduler configuration.")
        bad_conf = [t_start.utc.datetime] + ['unknown' for f in fields]
        conf = pd.DataFrame(bad_conf, columns=['time'] + fields)
        conf.set_index('time', inplace=True)
        conf.index = conf.index.tz_localize("UTC")
        
    conf['classname'] = "Scheduler configuration"
    # To get the scheduler relevant info in a single line, pull in ts_config_ocs
    # to the configuration information.
    ts_config_ocs_in_place = []
    for time in conf.index:
        prev_obsenv = obsenv.query('index < @time')
        if len(prev_obsenv) == 0:
            ts_config_ocs_in_place.append('Unknown')
        else:
            ts_config_ocs_in_place.append(prev_obsenv.iloc[-1]['ts_config_ocs'])
    conf['ts_config_ocs'] = ts_config_ocs_in_place
    def build_compact_description_string(x):
        desc_string = x.configurations.split(',')[-1]  + "<br> ts_config_ocs " +  x.ts_config_ocs
        link = f"https://github.com/lsst-ts/ts_config_ocs/tree/{x.ts_config_ocs}/Scheduler/feature_scheduler"
        url = f'<a href="{link}" target="_blank" rel="noreferrer noopener">{desc_string}</a>'
        return url
    conf['description'] = conf.apply(build_compact_description_string, axis=1)
    conf.rename({'configurations': 'config'}, axis=1, inplace=True)
    conf['script_salIndex'] = -1

    # Combine results
    sched_config = pd.concat([deps, conf, obsenv])

    # Reformat
    cols = ['classname', 'description', 'config', 'salIndex', 'script_salIndex']
    drop_cols = [c for c in sched_config.columns if c not in cols]
    sched_config.drop(drop_cols, axis=1, inplace=True)
    sched_config.sort_index(inplace=True)
    sched_config['timestampProcessStart'] = sched_config.index.copy().tz_localize(None).astype('datetime64[ns]')
    sched_config['finalScriptState'] = "Configuration"
    print(f"Found {len(sched_config)} scheduler configuration records")
    return sched_config

dd = await get_scheduler_configs(early_setup, sunrise, efd_client, obsenv_client, queueIndex=salindex)
cols = dd.columns
display(HTML(dd[cols].to_html(escape=False)))

print(dd.iloc[-1]['description'])


# What other BLOCKS have been requested in this night (outside the FBS)
display(Markdown("### JSON BLOCKS"))
#topic = 'lsst.sal.Scheduler.command_addBlock'
topic = 'lsst.sal.Scheduler.logevent_blockStatus'
fields = await efd_client.get_fields(topic)
fields = [f for f in fields if 'private' not in f]
dd = await efd_client.select_time_series(topic, fields, early_setup, sunrise, index=salindex)
if len(dd) == 0:
    print(f"No JSON BLOCKS added between {early_setup.iso} and {sunrise.iso}")
else:
    grouped_dd = dd.groupby('id')[['id', 'definition', 'executionsCompleted', 'hash', 'salIndex']].agg('max')
    grouped_dd_start = dd.groupby('id')[['id', 'definition', 'executionsCompleted', 'hash', 'salIndex']].agg('min')
    grouped_dd['night_executions']  = grouped_dd['executionsCompleted'] - grouped_dd_start['executionsCompleted']
    display(grouped_dd[['id', 'definition', 'hash', 'executionsCompleted', 'night_executions']])

In [ ]:
# Your JIRA Cloud base URL for API
ZEPHYR_BASE_URL = "https://api.zephyrscale.smartbear.com/v2/" + "testcases/"
JIRA_BASE_URL = "https://rubinobs.atlassian.net/rest/api/2/" + "issue/"

testcase_base_url = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/v2/testCase/"

jiraticket_base_url = "https://rubinobs.atlassian.net/browse/"

if len(dd) > 0:
    display(Markdown("For more information of test cases and blocks:"))
    for block in grouped_dd.id:
        if block is not None and block.startswith("BLOCK-T"):
            url = testcase_base_url + block
        elif block is not None and block.startswith("BLOCK-"):
            url = jiraticket_base_url + block
        display(Markdown(f'<a href="{url}" target="_blank" rel="noreferrer noopener">{block}</a>'))

<a id="EFD_targets"></a>

## EFD Targets and Observations

In [ ]:
# Fetch requested targets
topic = 'lsst.sal.Scheduler.logevent_target'
all_fields = await efd_client.get_fields(topic)
#print(all_fields)
targets = await efd_client.select_time_series(topic, all_fields, sunset, sunrise, index=salindex)

def demangle_note_old(x):    
    # remove _expnum
    x.target = copy.deepcopy(x.note)
    if "IM" in x.note:
        x.note = x.note.split(":")[1].split("_")[0]
    if 'spec' in x.note:
        x.note = 'HD' + x.note.split('HD')[-1]
    return x

# New note name -- still only note 
# But now note doubles for block + target name
def demangle_note(x):    
    vals = x.note.split(":")
    block = x.note.split(":")[0]
    if len(vals) > 1:
        target_name = x.note.split(':')[1]
        if "HD" in target_name: 
            # spec target
            pass
        elif "_" in target_name: 
            # most likely image target with tiles
            # for compactness, remove tile
            target_name = target_name.split("_")[0]
        x.science_program = block
        x.target_name = target_name
        x.note = target_name
    else:
        x.science_program = block
        x.target_name = block
        x.note = block
    if x.note == "BLOCK-305":
        x.note = 'cwfs'
    return x

if len(targets) == 0:
    display(Markdown(f"On night {day_obs} {telescope} recorded {len(targets)} target log events."))
    display(Markdown(f"This was for times between {sunset.iso}, {sunrise.iso}"))

    # debug help -- 
    targets = await efd_client.select_top_n(topic, all_fields, 2, index=salindex)
    display(Markdown("The most recent targets were recorded were:"))
    display(targets[['SchedulerID', 'airmass', 'ra', 'decl', 'skyAngle', 'exposureTimes0', 'skyBrightness', 'slewTime', 'note', 'salIndex']])
    targets = []

else:

    # target timestamp in EFD = time that the target is pushed to scriptqueue
    # this should be at the start of the previous observation
    targets["target_time"] = targets.index.copy()
    targets["target_mjd"] = Time(targets.index).mjd
    # estimate what time this target should be observed (start of observation)
    # == target_time + previous exposure time (?) + slew time
    cols = [c for c in targets if 'exposureTimes' in c]
    targets['total_exptime'] = targets[cols].sum(axis=1)
    
    # Sometimes there are targets which don't correspond to a real target
    # These seem to be triggered every time the Scheduler comes to ENABLED
    targets = targets.query('total_exptime > 0')
    
    previous_exposure_time = np.concatenate([np.array([0]), targets['total_exptime'][:-1]])
    # The target is supposed to be issued to the EFD when it hits the top of the scriptqueue 
    # which is supposed to be when the previous observation starts ... however
    # an observation having many scripts may mean it doesn't hit the top of the scriptqueue then.
    # (so this is probably more like a range of target.mjd + slewtime --- target + slewtime + previous exposure
    targets["previous_exptime"] = previous_exposure_time
    targets["target_obsmjd"] = Time(targets.index).mjd  + (targets['slewTime'])*seconds_to_days 

    # When estimating obsmjd expected from target -- include the previous
    # exposure time or not? (in theory, should. in practice, queue is busy so not including it can be helpful).
    include_prev_exptime = True

    if include_prev_exptime:
        targets["target_obsmjd"] += (targets["previous_exptime"])*seconds_to_days
    
    # demangle the note 
    targets['orig_note'] = targets.note.copy()
    targets['target_name'] = ''
    targets['block'] = ''
    targets = targets.apply(demangle_note, axis=1)
    
    targets = targets.sort_values(by='target_obsmjd')
    targets['target_id'] = np.arange(0, len(targets), 1)
    targets = targets.reset_index(drop=True)


    targets.reset_index()
    
    display(Markdown(f"On night {day_obs} {telescope} recorded {len(targets)} `target` events for \n{targets.target_name.unique()}"))


# Check how many snapshots too as this equals number of calls
topic = "lsst.sal.Scheduler.logevent_largeFileObjectAvailable"
fields = ["url"]
snapshots = await efd_client.select_time_series(topic, fields, sunset, sunrise, index=salindex)
if len(snapshots) > 0:
    display(Markdown(f"There were {len(snapshots)} snapshots recorded"))

In [ ]:
#display(HTML(targets[['target_id', 'target_obsmjd', 'ra', 'decl', 'slewTime', 'filter', 'exposureTimes0', 'note', 'orig_note']].to_html()))

In [ ]:
#display(HTML(pd.concat([snapshots, targets.set_index('target_time')[['target_mjd', 'ra', 'decl', 'filter', 'skyAngle']]]).sort_index().to_html()))
#display(HTML(targets[['target_mjd', 'ra', 'decl', 'filter', 'skyAngle', 'slewTime']].to_html()))

In [ ]:
# Observations are the completed observation script .. to be compared with visits from the consdb

topic = 'lsst.sal.Scheduler.logevent_observation'
all_fields = await efd_client.get_fields(topic)
obs = await efd_client.select_time_series(topic, all_fields, sunset, sunrise, index=salindex)

if len(obs) == 0:
    display(Markdown(f"On night {day_obs} {telescope} recorded {len(obs)} observation log events."))

else:
    # timestamp is the time of successful end of observation/JSON block
    obs['obs_time'] = obs.index.copy()
    obs['obs_mjd'] = Time(obs.index).mjd 

    # obs time - exposure time should be start of observation (if exposure time is correct)
    # Looking at values recorded, exptime * nexps is not total exposure time (exptime alone is)
    obs['obs_obsmjd'] = Time(obs.index).mjd - (obs['exptime'] * seconds_to_days)
    
    obs = obs.sort_values(by='obs_obsmjd')
    obs['obs_id'] = np.arange(0, len(obs), 1)
    obs = obs.reset_index(drop=True)
    
    display(Markdown(f"On night {day_obs} {telescope} recorded {len(obs)} `observation` events."))

In [ ]:
#display(HTML(obs[['ra', 'decl', 'exptime', 'filter', 'mjd', 'nexp', 'salIndex', 'targetId', 'obs_mjd',]].to_html()))

### Matching `targets` to `observations`

In [ ]:
def match_obs_and_targets(obs, targets):
    # This should be carried in the efd information - observation has a field for targetId
    if len(targets) == 0:
        print("No targets")
        return obs, targets
    if len(obs) == 0:
        print("No observations")
        targets['obs_id'] = -1
        return obs, targets
        
    # Check targets -> observations
    target_to_obs_match = np.zeros(len(targets), int) - 1
    obs_to_target_match = np.zeros(len(obs), int) - 1
    
    print("Matching observations against targets")
    count = 0

    # I'm finding that the first target can be very much delayed
    # so let's treat that one separately
    
    for i, (ri, t) in enumerate(targets.iterrows()):
        # Match obs_mjd, ra/dec/filter from target+observation
        # the target.obs_mjd may overestimate actual mjd by ~ previous exposure
        # if slewtime is inaccurate, target_obs.mjd may be inaccurate
        # if visit includes JSON BLOCK with many steps, could be delays 
        # so that obs_obsmjd is later than reality
        # some targets may never get completed observations
        slew_error = 3.0 * minutes_to_days
        target_error = t.previous_exptime * seconds_to_days
        random_error = 14 * minutes_to_days
        # don't match to a repeat of the same target much later
        X = 30 * minutes_to_days
        # CWFS target matching should try to use nearest time rather than ra/dec ? 
        if t.note == 'BLOCK-305':
            delta_t = np.abs(t.target_obsmjd - obs.obs_obsmjd)
            match = np.where((delta_t - delta_t.min() < 1e-2) # closest in time
                             & ((obs.obs_obsmjd - t.target_obsmjd) <= X) # not more than X
                             & (t['filter'] == obs['filter'])
                             & (t['total_exptime'] == obs['exptime'])
                             & (obs_to_target_match == -1))[0]
        else:
            match = np.where((np.abs(t.target_obsmjd - obs.obs_obsmjd) < target_error + slew_error + random_error) # close in time
                             & ((obs.obs_obsmjd - t.target_obsmjd) <= X) # not more than X
                             & (t.ra == obs.ra) 
                             & (t.decl == obs.decl) 
                             & (t['filter'] == obs['filter'])
                             & (obs_to_target_match == -1))[0]

        if len(match) == 0:
            print(f'no obs match for target {t.target_id} {t.note}')
            count += 1
            target_to_obs_match[i] = -1
        else:
            # Consider target to have found the best match 
            # in the first (soonest) of 'match'
            idx = match[0]
            target_to_obs_match[i] = obs.iloc[idx]['obs_id']
            # And consider observation to have found a match
            obs_to_target_match[idx] = t['target_id']

    obs['target_id'] = np.array(obs_to_target_match)
    targets['obs_id'] = np.array(target_to_obs_match)
    
    print(f'failed to match {count} targets to observations')
    print(f'compare to {len(targets) - len(obs)} expected difference')
    
    return obs, targets

In [ ]:
# Short columns helpful for cross-matching tests
target_cols = ['ra', 'decl', 'skyAngle', 'filter', 'total_exptime', 'note', 'target_obsmjd']
obs_cols  = ['ra', 'decl', 'rotSkyPos', 'filter', 'exptime', 'nexp', 'obs_obsmjd',]

In [ ]:
obs, targets = match_obs_and_targets(obs, targets)

In [ ]:
# Targets which did not get observations
if len(targets) > 0:
    tt = targets.query('obs_id == -1')[['ra', 'decl', 'skyAngle', 'filter', 'total_exptime', 'note', 'orig_note', 'target_mjd', 'obs_id',]]
    if len(tt) > 0:
        display(Markdown("The following `targets` were not able to be linked with `observations`"))
        display(tt)
    else:
        display(Markdown("All logged `targets` were linked with `observations`"))

In [ ]:
if len(obs) > 0:
    oo = obs.query('target_id == -1')[['ra', 'decl', 'rotSkyPos', 'filter', 'exptime', 'obs_mjd', 'target_id']]
    if len(oo) > 0:
        display(Markdown("The following `observations` were unable to be matched with `targets` (this is unusual)."))
        display(oo)
    else:
        display(Markdown("All `observations` were linked with `targets`"))

In [ ]:
# Join target and observations into one dataframe
if len(obs) > 0:
    # Join targets + obs
    jj = obs.query('target_id > -1').join(targets, on='target_id', lsuffix='_o', rsuffix='_t')
    jj = jj.sort_values(by='obs_obsmjd')
    
    jj['delta_obs'] = np.concatenate([np.array([0]), np.diff(jj['obs_obsmjd'])/minutes_to_days])  # minutes -- should be close to expected overhead
    jj['delta_request'] = (jj['obs_obsmjd'] - jj['target_obsmjd']) / minutes_to_days  # minutes  -- should be close to 0
    jj['expected_gap'] = (jj['previous_exptime']+ jj['slewTime'])/60  # minutes - expected overhead from models
    anomalous_overhead = jj['delta_obs'].values - jj['expected_gap'].values
    anomalous_overhead[0] = 0
    jj['anomalous_overhead'] = anomalous_overhead
else:
    jj = []

<a id="ConsDb_visits"></a>

## ConsDb Visit Information

In [ ]:
# Add consdb

day_obs_int = int(day_obs.replace('-', ''))

visit_query = f'''
    SELECT * FROM cdb_{instrument}.visit1
     where day_obs = {day_obs_int}
'''

quicklook_query = f'''
    SELECT q.*  FROM cdb_{instrument}.visit1_quicklook as q,
    cdb_{instrument}.visit1 as v
     WHERE v.day_obs = {day_obs_int} and q.visit_id = v.visit_id
'''

# Use the ConsDB Client, and add a couple of tries 
consdb = ConsDbClient()

# Ugh, wrap the whole thing again in case the consdb is just down
try: 
    try:
        visits = consdb.query(visit_query).to_pandas()
    except requests.HTTPError or requests.JSONDecodeError:
        # Try twice
        visits = consdb.query(visit_query).to_pandas()
    
    quicklook = consdb.query(quicklook_query).to_pandas()
except requests.HTTPError:
    display(Markdown("ConsDB seems to be unreachable"))
    visits = []
    quicklook = []

if len(visits) > 0:
    display(Markdown(f"Retrieved {len(visits)} visits from consdb"))
    obj_visits = visits.query('img_type == "OBJECT"')
    display(Markdown(f"{len(obj_visits)} of these are `OBJECT` images"))

if len(quicklook) > 0:
    visits = visits.join(quicklook, on='visit_id', lsuffix='', rsuffix='_q')
    visits = visits.copy()
    display(Markdown(f"And added quicklook stats"))

if len(visits) == 0:
    display(Markdown(f"No visits for {telescope} on {day_obs} retrieved from consdb"))

# Patch science_program if was None
values = (dict([[e,""] for e in ['science_program','target_name', 'observation_reason']]))
visits.fillna(value=values, inplace=True)

In [ ]:
verbose = False
short_cols = ['seq_num', 'obs_start_mjd', 'obs_end_mjd', 'exp_time', 'shut_time', 'dark_time', 's_ra', 's_dec', 'band', 'airmass', 
              'img_type', 'target_name', 'science_program', 'observation_reason', 'dimm_seeing']
if len(visits)>0 and verbose:
    display(visits[short_cols])

In [ ]:
# construct a 'note' to match visits with the target/observation colors
def construct_note_old(x):
    if x.science_program == 'cwfs' or x.science_program == 'cwfs-focus-sweep':
        note = 'cwfs'
    elif x.science_program == "BLOCK-305":
        note = 'cwfs'
    elif x.science_program == "BLOCK-295":
        note = 'calibrations'
    elif x.target_name == 'FlatField position':
        note = 'calibrations'
    else:
        note = x.target_name
    if len(note) == 0:
        note = 'unknown'
    elif x.science_program == 'BLOCK-306':
        note = x.target_name
    elif x.science_program == 'spec-survey':
        note = x.target_name
    else:
        note = 'unknown'
    return note

def construct_note(x):
    if x.science_program == 'cwfs' or x.science_program == 'cwfs-focus-sweep' or x.science_program == "BLOCK-305":
        note = 'cwfs'
    elif x.img_type == 'ACQ':
        note = 'acq'
    elif x.science_program == "BLOCK-295":
        note = 'calibrations'
    elif x.target_name == 'FlatField position':
        note = 'calibrations'
    else:
        if len(x.target_name) > 0: 
            note = x.target_name
        elif len(x.science_program) > 0: 
            note = x.science_program
        else:
            note = x.observation_reason
        if note is None:
            note = 'unknown'
    return note

if len(visits)>0:
    note = visits.apply(construct_note, axis=1)
    visits['note'] = note
    display(Markdown(f"ConsDB constructed note names: {visits.note.unique()}"))

In [ ]:
# Link consdb visits with targets .. note that this can be many visits -> one target
# Also, should match against targets and not observations, because there can be some visits from an incomplete target

with warnings.catch_warnings():
    # probably shouldn't entirely suppress this, but can't quite find PerformanceWarning source
    warnings.simplefilter('ignore')
    if len(visits) > 0:
        visit_target_id = np.zeros(len(visits)) - 1
        if len(targets) > 0:
            for note in targets.note.unique():
                vv = visits.query('note == @note and science_program != None')
                if len(vv) == 0:
                    continue
                tt = targets.query('note == @note')
                # Find sequential visits 
                gaps = [[s, e] for s, e, in zip(vv.index, vv.index[1:]) if s+1 < e]
                left = [vv.index[0]] + [g[1] for g in gaps]
                right = [g[0] for g in gaps] + [vv.index[-1]]
                consecutive = list(zip(left, right))
                for l, r in zip(left, right):
                    # if there are repeated targets with the same note, resulting in
                    # sequential visits to the same note ... this will match all 
                    # the visits to the first target in the series (unfortunately)
                    delta_t = vv.loc[l]['obs_start_mjd'] - tt['target_mjd']
                    idx = np.where(delta_t > 0, delta_t, np.inf).argmin()
                    visit_target_id[l:(r+1)] = tt.iloc[idx]['target_id']
            
        visits['target_id'] = visit_target_id

    
        display(Markdown(f"Matched {len(visits.query('target_id > 1'))} `visits` to `targets`, out of a total of {len(visits)} visits."))
        remainder = visits.query('target_id == -1')
        display(Markdown(f"The remaining {len(remainder)} visits were of type {remainder.img_type.unique()}"))
                         #f" with science_programs {remainder.science_program.unique()}, observation_reason {remainder.observation_reason.unique()}."))

# Matched!  .. could now compare time of actual visit with time of target and time of observation, try to sort out if there are timing issues

In [ ]:
c = None
if len(visits) > 0:
    groupcols = ['science_program', 'img_type', 'target_name', 'observation_reason', 'day_obs', 'visit_id'] 
    c = visits[groupcols].groupby(['science_program', 'img_type']).agg({'science_program' : ['first'],
                                                                        'target_name' : ['unique'], 
                                                                        'observation_reason' : ['unique'],
                                                                        'day_obs' : ['nunique'],
                                                                        'visit_id' : ['first', 'last', 'count']})
    display(Markdown(f"ConsDB Visits"))
    with option_context('display.max_colwidth', None):
        display(HTML(c.sort_values(by=('visit_id', 'first')).to_html()))

In [ ]:
# This is a neat trick to dynamically create a download link .. but probably not necessary here 

# import base64

# def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
#     csv = df.to_csv()
#     b64 = base64.b64encode(csv.encode())
#     payload = b64.decode()
#     html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
#     html = html.format(payload=payload,title=title,filename=filename)
#     return HTML(html)

# display(create_download_link(targets, title=f"Download linked targets CSV ({day_obs})"))
# display(create_download_link(obs, title=f"Download linked observations CSV ({day_obs})"))
# display(create_download_link(visits, title=f"Download linked visits CSV ({day_obs})"))

<a id="Overheads"></a>

## Overheads and Issues

Comparing the expected overhead from the predicted slewtime and exposure time to the actual time recorded between observation events (where matched to targets). 
This can be used to identify longer-than-expected overheads between visits. 
Currently, this doesn't use the Visit information.

In [ ]:
overhead_error = 3
typical_anomalous_overhead = 0

if len(jj) > 0:
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))
    _ = ax[0].hist(jj['delta_obs'], bins=np.arange(-2, 25, 0.5), alpha=0.6, histtype='bar', label='delta observation obsmjd')
    _ = ax[0].hist(jj['expected_gap'], bins=np.arange(-2, 25, 0.5), alpha=0.6, histtype='bar', label='predicted gaps between obs')
    ax[0].legend(loc=(0.45, 0.85))
    ax[0].set_xlabel("Minutes", fontsize='large')

    _ = ax[1].hist(jj['anomalous_overhead'], bins=50, histtype='bar', label="(delta_obs) - (pred_gap)")
    ax[1].axvline(overhead_error, color='r', linestyle=':')
    ax[1].legend(loc=(0.58, 0.83))
    ax[1].set_xlabel("Anomalous Overhead (minutes)", fontsize='large')

    typical_anomalous_overhead = np.median(jj.query('anomalous_overhead < @overhead_error')['anomalous_overhead'])
    display(Markdown(f"Median value of anomalous overhead, not including values beyond dashed line:  {typical_anomalous_overhead.round(2)} minutes"))

In [ ]:
# Where did the time between observations just take longer than expected 
# delta obs compared to (slewTime + exptime) BUT sheduler target estimate was correct

breaks = []
if len(jj)>0: 
    overheads = np.where(jj.anomalous_overhead > overhead_error)[0]
    
    for b in overheads:
        b_start = jj.iloc[b-1]['obs_obsmjd'] + jj.iloc[b-1]['exptime']/60/60/24
        b_end = jj.iloc[b]['obs_obsmjd'] #- jj.iloc[b]['slewTime']/60/60/24
        # look for cwfs sweeps around the break, as they seem to mess with timings
        b_min = np.max([b-1, 0])
        b_max = np.min([b+1, len(jj)-1])
        check_for_cwfs = jj.iloc[b_min:b_max+1].query("note == 'cwfs'")
        if len(check_for_cwfs) > 0 and (b_end - b_start) < 10 * minutes_to_days:
            # cwfs sweeps have variable timing and can disturb the expected timings
            # but also, if we did have a long break, then count these anyway.
            pass
        else:
            breaks.append([b_start, b_end])
    
    print(f"There are {len(overheads)} points where the time between observations doesn't match predicted overheads within {overhead_error} minutes.",
           f" CWFS visits are indicated in {len(overheads) - len(breaks)} of these breaks, and have a variable effect on the overhead so will be discounted.")
    print()
    
    display(Markdown("Details of unexpected deltas in targets-observations (prev - unexpected - next)"))
    for o in overheads:
        display(jj[['target_id', 'obs_id_t', 'target_name', 'delta_obs', 'expected_gap', 'obs_obsmjd', 'target_obsmjd']][o-1:o+2])


In [ ]:
if len(jj) > 0:
    plt.figure()
    
    eps = 1
    for note in jj.note.unique():
        j = jj.query('note == @note')
        plt.plot(j.delta_obs, j.expected_gap, '.', label=note)
    j = jj.iloc[overheads]
    plt.plot(j.delta_obs, j.expected_gap, 'o',
             markersize=10, color='k', markerfacecolor='none', label='Big Overhead')
    plt.legend(loc=(1.01, 0.5))
    for exptime in jj.exptime.unique():
        plt.axhline(exptime/60, color='gray', linestyle=':', alpha=0.6)
    
    x = np.arange(0, 60)
    plt.plot(x, x, 'r:')
    plt.fill_between(x, x-overhead_error, x+overhead_error,  color='r', alpha=0.1)
    plt.grid(True, alpha=0.3)

    max_x = np.max(jj.delta_obs) + eps
    max_y = np.max(jj.expected_gap) + eps/2
    if max_x > max_y + 10:
        max_x = max_y + overhead_error
    plt.xlim(0, max_x)
    plt.ylim(0, max_y)
    
    plt.xlabel("DeltaT between start of observations (minutes)")
    plt.ylabel("Expected gap (exptime + slew) (minutes)")    

In [ ]:
# Identify any sequences of incomplete targets, to color-code later
breaks_targets = []
if len(targets) > 0:
    missing_break = 0
    t = targets.query('obs_id == -1').index.values
    if len(t) > 0:        
        gaps = [[s, e] for s, e, in zip(t, t[1:]) if s+1 < e]
        left = [t[0]] + [g[1] for g in gaps]
        right = [g[0] for g in gaps] + [t[-1]]
        consecutive = list(zip(left, right))
        # Trim off the end, as a likely single-missed target
        if consecutive[-1][0] == consecutive[-1][1]:
            consecutive = consecutive[:-1]
        for c_visits in consecutive:
            start_idx = c_visits[0]
            end_idx = c_visits[1] + 1
            if end_idx > start_idx + 1: 
                # Use time target recorded at as the start
                b_start = targets.iloc[start_idx]['target_mjd']
                # Use time target would have expected observation as end point
                if end_idx > len(targets) - 1:
                    end_idx = end_idx - 1
                b_end = targets.iloc[end_idx]['target_obsmjd']
                breaks_targets.append([b_start, b_end])
                missing_break += 1

    if missing_break == 1:
        display(Markdown(f"There was {missing_break} period of consecutive incomplete `targets`. Could be due to weather or script failures."))
    if missing_break > 1:
        display(Markdown(f"There were {missing_break} periods of consecutive incomplete `targets`. Could be due to weather or script failures."))
    if missing_break == 0:
        display(Markdown("There were no periods of consecutive incomplete `targets`."))


## Time Summary

In [ ]:
# Try summing up time into different buckets, although it should be noted that this is still in need of more validation

# First find when the ScriptQueue was running and enabled .. this could be slightly different than when Targets are sent.
topic = 'lsst.sal.ScriptQueue.logevent_queue'
fields = ['running', 'enabled', 'currentSalIndex', 'salIndices0', 'pastSalIndices0']
dd = await efd_client.select_time_series(topic, fields, civil_sunset, sunrise)

if len(dd) > 0:
    # Parse out simonyi / auxtel / ocs entries from the scriptqueue
    query = f"pastSalIndices0.astype('str').str.startswith('{salindex}')"
    dd = dd.query(query)

if len(dd) > 0:
    dd['go'] = (dd['running'] & dd['enabled']).values
    dd['go_prev'] = np.concatenate([np.array([False]), dd['go'].values[:-1]])
    start = dd.query('(go != go_prev) and (go)').index.values
    end = dd.query('(go != go_prev) and not (go)').index.values
    if len(end) == len(start) - 1:
        end = np.concatenate([end, dd.index[-1:].values])
    start_queue = Time(start).mjd
    end_queue = Time(end).mjd
    time_queue_active = (end_queue - start_queue).sum()
    sunset_to_queue = (start_queue[0] - sunset.mjd)
else:
    time_queue_active = 0
    sunset_to_queue = night_length


# Need to find time that the FBS is enabled and running


# Let's assume we found the breaks accurately ... but also try not to double count
tt = np.arange(sunset.mjd, sunrise.mjd, 1/60/24)
val = np.zeros(len(tt))
for b in breaks:
    val = np.where((tt >= b[0]) & (tt <= b[1]), 1, val)
for b in breaks_targets:
    val = np.where((tt >= b[0]) & (tt <= b[1]), 1, val)
time_in_breaks = val.sum() * minutes_to_days

# Also look at when targets were being sent for this telescope
if len(targets) > 0:
    last_target = targets.iloc[-1]['target_mjd']
    first_target = targets.iloc[0]['target_mjd']
    time_with_targets = (last_target - first_target) - time_in_breaks
    sunset_to_targets = (first_target - sunset.mjd)
else:
    time_with_targets = 0
    sunset_to_targets = night_length

# Predicted overhead in slews + anomalous overhead
if len(jj) > 0:
    time_overhead = (jj['slewTime'].sum() + jj['anomalous_overhead'].sum()) * seconds_to_days
else:
    time_overhead = 0


# Count up observing time per science_program during the night
# Just roll up OBJECT, CWFS, ACQ and FOCUS .. 
night_visits = 0
if len(visits) > 0:
    visits_after_sunset = visits.query("obs_start_mjd > @sunset.mjd")
    night_visits = len(visits_after_sunset)
    if night_visits > 0:
        programs = visits_after_sunset.science_program.unique()
        onsky_time = {}
        for p in programs:
            onsky_time[p] = visits_after_sunset.query('science_program == @p').exp_time.sum() * seconds_to_days
        # Consolidate "random" programs .. while getting metadata sorted
        if '' in onsky_time:
            onsky_time['Unknown'] = onsky_time['']
            del onsky_time['']
        sunset_to_visits = (visits_after_sunset.iloc[0].obs_start_mjd - sunset.mjd) 
        civilsunset_to_visits = (visits_after_sunset.iloc[0].obs_start_mjd - civil_sunset.mjd) 
if night_visits == 0:
    sunset_to_visits = night_length
    civilsunset_to_visits = night_length
    onsky_time = {}

dd = pd.DataFrame([night_length, time_queue_active, time_with_targets, 
                   sunset_to_queue, sunset_to_targets, sunset_to_visits, civilsunset_to_visits,
                   time_in_breaks, time_overhead,], 
                  index=["Total night time", "Time ScriptQueue Active", "Time with Targets",
                         "Sunset to Scriptqueue", "Sunset to Targets", "Sunset to Visits", "Civil Sunset to Visits",
                         "Possible issue time", "Predicted Overheads", 
                         ], 
                  columns=[day_obs])
dd_obs = pd.DataFrame(onsky_time, index=[day_obs])
# convert days (above) to hours
dd = (dd * 24).T
dd_obs = (dd_obs * 24)

display(Markdown("Prototyping summary of time accounting, WIP"))
display(Markdown("Units: Hours. Sunset: -12 degree."))
display(dd.round(2))
if len(visits) > 0:
    display(Markdown("Exposure time per science_program"))
    display(dd_obs.round(2))
else:
    display(Markdown("No consdb visits; not counting time per science_program"))

dd = dd.join(dd_obs)
if dd["Time ScriptQueue Active"].iloc[0] > 0:
    fig, ax = plt.subplots(1, 2, figsize=(15, 8))
    fig.tight_layout(pad=17)
    if len(visits) > 0:
        cols = list(onsky_time.keys()) #+ ["Possible issue time", "Overheads", ]
        plot_data = dd.loc[:, cols] / dd.iloc[0, 1]
        plot_data.plot(kind='bar', stacked=True, ax=ax[0],
                       ylim=[0, 1],
                       legend=False, ylabel='Fraction of Time with Active ScriptQueue').legend(loc=(1.01, 0.2))
        ax[0].grid(True, alpha=0.3)
    
    cols = list(onsky_time.keys()) + ["Time with Targets", "Time ScriptQueue Active", "Total night time"]
    plot_data = dd.loc[:, cols]
    plot_data
    plot_data.plot(kind='bar', stacked=False, ax=ax[-1],
                   legend=False, ylabel='Hours').legend(loc=(1.01, 0.2))
    ax[-1].grid(True, alpha=0.3)

<a id="Summary_plot"></a>

## Summary Plot of FBS Targets, Observations, and ConsDB Visits

In [ ]:
def mjd_to_datetime(mjd, timezone=tz):
    return Time(mjd, format='mjd', scale='utc').to_datetime(timezone=timezone)

In [ ]:
eps = 1
fig, ax = plt.subplots(figsize=(11, 7))
ax_utc = ax.twiny()

ax.set_title(f"{telescope} DAYOBS {day_obs}", pad=20)


# Shade astronomical events
ax.fill_between([mjd_to_datetime(night_events['sun_n12_setting']), 
                  mjd_to_datetime(night_events['sun_n18_setting'])],
                 2.5, 0.0, color='lightgray', alpha=0.3)
ax.fill_between([mjd_to_datetime(night_events['sunset']), 
                 mjd_to_datetime(night_events['sun_n12_setting'])], 
                  2.5, 0.0, color='gray', alpha=0.3)
ax.fill_between([mjd_to_datetime(night_events['sun_n18_rising']), 
                 mjd_to_datetime(night_events['sun_n12_rising'])],
                 2.5, 0.0, color='lightgray', alpha=0.3)
ax.fill_between([mjd_to_datetime(night_events['sun_n12_rising']), 
                 mjd_to_datetime(night_events['sunrise'])],
                 2.5, 0.0, color='gray', alpha=0.3)

if not np.isnan(night_events['moonrise']):
    ax.axvline(mjd_to_datetime(night_events['moonrise']), linestyle=':', color='blue', alpha=0.3)
if not np.isnan(night_events['moonset']):
    ax.axvline(mjd_to_datetime(night_events['moonset']), linestyle=':', color='red', alpha=0.3)

plot_fbs = True

plot_visits = True
if len(visits) == 0:
    plot_visits = False

# Assign distinct target sets with different colors
if len(jj) > 0:
    notes = jj.note.unique()
    note_colors = {}
    for i, n in enumerate(notes):
        note_colors[n] = cc.glasbey[i]
elif len(visits) > 0:
    notes = visits.note.unique()
    note_colors = {}
    for i, n in enumerate(notes):
        note_colors[n] = cc.glasbey[i]
else: # no completed observations, no visits, but targets
    notes = targets.note.unique()
    note_colors = {}
    for i, n in enumerate(notes):
        note_colors[n] = cc.glasbey[i]

if plot_fbs and len(jj) > 0:
    # Plot the successfully recorded 'observation' scripts
    for note in jj.note.unique():
        j = jj.query('note == @note')
        ax.plot(mjd_to_datetime(j.obs_obsmjd), j.airmass, '.', 
                markersize=9,  color=note_colors[note], label=f"Target+Obs {note}")

    # Shade breaks in observation events
    if len(breaks) > 0:
        for break_count, b in enumerate(breaks):
            ax.fill_between(mjd_to_datetime(b), 2.5, 0.0, color='pink', alpha=0.3)
            ax.text(mjd_to_datetime(b[0]), 0.97, f"D-{break_count}")

    # Plot the observations that come after what looks like big overheads
    #j = jj.iloc[overheads]
    #plt.plot(j.obs_obsmjd, j.airmass, 'o', 
    #         markersize=10, color='k', markerfacecolor='none', label='Big Overheads')

if plot_fbs and len(targets) > 0:

    snapshot_times = Time(snapshots.index.values).to_datetime()
    ax.plot(snapshot_times, np.ones(len(snapshots)) * 1.01, color='orange', 
            marker='|', markersize=13, linestyle='')
    
    # Plot the targets which did not get 'observations' recorded
    incomplete_targets = targets.query('obs_id == -1')
    for note in incomplete_targets.note.unique():
        try:
            color = note_colors[note]
        except KeyError:
            color = cc.glasbey[i]
            i -= 1
        t = incomplete_targets.query('note == @note')
        ax.plot(mjd_to_datetime(t.target_obsmjd), 
                 t.airmass, '+', markersize=11,
                 label=f'TargetOnly {note}', color=color, zorder=1)

    # Shade incomplete target streaks
    if len(breaks_targets) > 0:
        for break_count, b in enumerate(breaks_targets):
            ax.fill_between(mjd_to_datetime(b), 2.5, 0.0, color='lightblue', alpha=0.3)
            ax.text(mjd_to_datetime(b[0]), 0.94, f"T-{break_count}")


    # Shade time before first targets 
    # ax.fill_between([mjd_to_datetime(sunset.mjd), mjd_to_datetime(targets.target_mjd[0])],
    #                2.5, 0.0, color='darkorange', alpha=0.1)


i = len(cc.glasbey) - 1
if plot_visits: 
    # Plot the visits recorded in the consdb 
    obs_visits = visits.query('img_type == "OBJECT"')
    ac_visits = visits.query('img_type == "ACQ"')
    focus_visits = visits.query('img_type == "FOCUS"')
    cwfs_visits = visits.query('img_type == "CWFS"')
    for note in visits.note.unique():
        try:
            color = note_colors[note]
        except KeyError:
            color = cc.glasbey[i]
            i -= 1
        # Label the first (in order) of these we find
        label = f'ConsDb {note}'
        v = obs_visits.query('note == @note')
        if len(v) > 0:
            ax.plot(mjd_to_datetime(v.obs_start_mjd), v.airmass, 'o', 
                     markersize=7, markerfacecolor='none', alpha=0.8,
                     label=label, color=color, zorder=3)
            label = None
        v = cwfs_visits.query('note == @note')
        if len(v) > 0:
            ax.plot(mjd_to_datetime(v.obs_start_mjd), v.airmass, 'H', 
                     markersize=8, markerfacecolor='none', alpha=0.5,
                     label=label, color=color, zorder=3)
            label = None
        v = ac_visits.query('note == @note')
        if len(v) > 0:
            ax.plot(mjd_to_datetime(v.obs_start_mjd), v.airmass, '*', 
                     markersize=9, markerfacecolor='none', alpha=0.5,
                     label=label, color=color, zorder=3)
            label = None
        v = focus_visits.query('note == @note')
        if len(v) > 0:
            ax.plot(mjd_to_datetime(v.obs_start_mjd), v.airmass, 'D', 
                     markersize=5, markerfacecolor='none', alpha=0.5,
                     label=label, color=color, zorder=3)
            label = None

ax.legend(loc=(1.01, 0.0), ncol=2)

x0 = night_events['sunset']+30/60/24

ax.set_xlim(mjd_to_datetime(night_events['sunset']+30/60/24), 
         mjd_to_datetime(night_events['sunrise']-30/60/24))
ax_utc.set_xlim(mjd_to_datetime(night_events['sunset']+30/60/24, timezone=tz_utc), 
         mjd_to_datetime(night_events['sunrise']-30/60/24, timezone=tz_utc))

ax.set_xlim(mjd_to_datetime(night_events['sunset']+30/60/24), 
         mjd_to_datetime(night_events['sunrise']-30/60/24))
ax_utc.set_xlim(mjd_to_datetime(night_events['sunset']+30/60/24, timezone=tz_utc), 
         mjd_to_datetime(night_events['sunrise']-30/60/24, timezone=tz_utc))

# Set ticks relevant sides
ax.tick_params(axis="x", bottom=True, top=False, labelbottom=True, labeltop=False)
ax_utc.tick_params(axis="x", bottom=False, top=True, labelbottom=False, labeltop=True)

# Rotate and align bottom ticklabels
plt.setp([tick.label1 for tick in ax.xaxis.get_major_ticks()], rotation=45,
         ha="right", va="center", rotation_mode="anchor")

# Rotate and align top ticklabels
plt.setp([tick.label2 for tick in ax_utc.xaxis.get_major_ticks()], rotation=45,
         ha="left", va="center",rotation_mode="anchor")

plt.grid(True, alpha=0.2)

plt.ylim(2.3, 0.9)

ax.set_ylabel("Airmass", fontsize="large")
ax.set_xlabel(f"Time ({tz})", fontsize="large")
ax_utc.set_xlabel("Time (UTC)", fontsize='large')
_ = plt.ylabel("Airmass", fontsize="large")

Visits from the ConsDB, as well as `Targets` and `Observations` from the EFD records generated from the FeatureBasedScheduler. <br>
The plot above stretches from sunset to sunrise, with civil, -12 degree and -18 degree sunset and sunrise indicated by the intensity of the gray shading. A blue (red) dashed line indicates the time of moonrise (moonset), if applicable. 

Blue shaded regions (if present) indicate periods of time with multiple sequential `Targets` that failed to find an expected `Observation` record in the EFD, and are numbered T-X. <br>
Pink shaded regions (if present) indicate larger than expected overheads between sequential `Observation` events, and are numbered D-X. <br> 
Periods with both a delay of target and a series of sequential target failures are shaded both pink and blue, which results in lavender regions (but these are not double-counted above in possible issue time). <br>
A orange shaded region (if present) on the left of the plot indicates the time from sunset until the time the first `Target` is sent from the FBS.

`Targets` which have been linked with a corresponding `Observation` event (indicating the observing script completed successfully) are indicated by solid circles. `Targets` which were not able to be linked to an `Observation` are shown by crosses. 

`Visits` for `OBJECT` images are indicated by open circles, while `ACQ` `visits` are indicated by stars; `FOCUS` `visits` are shown with diamonds while `CWFS` `visits` are shown by hexagons.

Both `Targets`, `Observations` and `Visits` are color-coded by their `note` values which are constructed in this notebook (using a combination of science_program and target_name). 

<a id="Night_report"></a>

## Night Report

In [ ]:
def usdf_requests(API_ENDPOINT, auth, params):
    # Try twice
    response = requests.get(API_ENDPOINT, auth=auth, params=params)
    if response.status_code != 200:
        response = requests.get(API_ENDPOINT, auth=auth, params=params)
    # Try dev as backup if still not working ..
    if response.status_code != 200:
        API_DEV = API_ENDPOINT.replace('usdf-rsp', 'usdf-rsp-dev')
        # Try twice
        response = requests.get(API_DEV, auth=auth, params=params)
        if response.status_code != 200:
            response = requests.get(API_DEV, auth=auth, params=params)
    if response.status_code != 200:
        print(f"{API_ENDPOINT} and {API_DEV} seem to be down.")
        messages = []
    else:
        messages = response.json()
    return messages

In [ ]:
if telescope.startswith("Aux"):
    tel_nr = "AuxTel"
else:
    tel_nr = "Simonyi"

this_dayobs = day_obs.replace('-', '')
next_dayobs = (Time(day_obs, format='iso') + TimeDelta(1, format='jd')).iso[0:10].replace('-', '')

params = {"telescopes" : tel_nr,
          "min_day_obs" : this_dayobs,
          "max_day_obs" : next_dayobs,
          "is_valid" : "true",
         }

token = get_access_token()
auth = ("user", token)

API_ENDPOINT = "https://usdf-rsp.slac.stanford.edu/nightreport/reports"

logs = usdf_requests(API_ENDPOINT, auth=auth, params=params)
    
if len(logs) == 0:
    print("No night report available.")


if len(logs) > 0:
    for log in [logs[0]]:
        display(Markdown(f"Observing crew : {log['observers_crew']}"))
        night_plan_block = "BLOCK" + urllib.parse.urlparse(log['confluence_url']).fragment.split("BLOCK")[-1]
        if night_plan_block == "BLOCK":
            night_plan_block = log['confluence_url']
        url = log['confluence_url']
        display(Markdown(f'Night plan : <a href="{url}" target="_blank" rel="noreferrer noopener">{night_plan_block}</a>'))
        display(Markdown("<strong>Summary</strong>"))
        display(Markdown(log['summary']))
        display(Markdown("<strong>Status</strong>"))
        display(Markdown(log['telescope_status']))